### 最大边界相关算法

用于计算查询文本和搜索文档之间的相似度，然后对文档进行排序。算法公式为$$\text{MMR}(Q,C,R) = Arg \max_{d_{i},in,c}^k [\lambda sim(Q,d_{i}) - (a- \lambda) \max_{d_{j},k} (sim(d_{i},d_{j}))]$$

其中Q指查询文本，C指搜索文档集合，R为一个已经球的的以相关度为基础的初始集合，$Arg \max_{d_{i},in,c}^k$表示搜索返回的K个句子的索引。

具体到文本摘要生成任务中，Q和C表示整篇文档，$d_{i}$表示文档中的某个句子，[ ]中第一项表示文档中某个句子和整篇文档的相似度，后一项表示文档中的某个句子和已经抽取的摘要句子的相似度。通过这样的方式，希望抽取的句子既能表达整个文档的含义，又可以具备多样性，$\lambda$表示两者重要性的权衡。

In [1]:
from sklearn.feature_extraction.text import CountVectorizer
from pprint import pprint
import operator



def encode_sen(sen,corpus):
    """
    input: sentence and corpus 
    output :  bag of words vector of sentence 
    """
    cv = CountVectorizer()
    cv = cv.fit(corpus)
    vec = cv.transform([sen]).toarray()
    return vec[0]

def cosin_distance(vector1, vector2):
    """
    input: two bag of words vectors of sentence  
    output :  the similarity between the sentence
   
    """
    dot_product = 0.0
    normA = 0.0
    normB = 0.0
    for a, b in zip(vector1, vector2):
        dot_product += a * b
        normA += a ** 2
        normB += b ** 2
    if normA == 0.0 or normB == 0.0:
        return None
    else:
        return dot_product / ((normA * normB) ** 0.5)


def doc_list2str(doc_list):
    """
    transform the doc_list to str 
    """
    docu_str = ""
    for wordlist in doc_list:
        docu_str += " ".join(wordlist)
    return docu_str


def MMR(doc_list,corpus):
    """
    input ：corpus and the docment you want to extract 
    output :the abstract of the docment 
    """
    Corpus = corpus
    docu = doc_list2str(doc_list)
    doc_vec = encode_sen(docu,Corpus)
    QDScore = {}
    ###calculate the  similarity of every sentence with the whole corpus
    for sen in doc_list:
        sen = " ".join(sen)

        sen_vec = encode_sen(sen,corpus)
        score = cosin_distance(sen_vec,doc_vec)
        QDScore[sen] = score

  
    n = 2
    alpha = 0.7
    Summary_set = []
    while n > 0:
        MMRScore = {}
        ### select the first sentence of abstract
        if Summary_set == []:
            selected = max(QDScore.items(), key=operator.itemgetter(1))[0]
            Summary_set.append(selected)

        Summary_set_str = " ".join(Summary_set)

        for sentence in QDScore.keys():
             #calculate MMR  
            if sentence not in Summary_set:
                sum_vec = encode_sen(Summary_set_str, corpus)
                sentence_vec = encode_sen(sentence,corpus)
                MMRScore[sentence] = alpha * QDScore[sentence] - (1 - alpha) * cosin_distance(sentence_vec,sum_vec)
        selected = max(MMRScore.items(), key=operator.itemgetter(1))[0]
        Summary_set.append(selected)
        n -= 1
    # print(len(Summary_set))
    return  Summary_set

In [7]:
import jieba
docment = "伴随着世界杯的落幕，俱乐部联赛筹备工作又成为主流，转会市场必然也会在世界杯的带动下风起云涌，不过对于在本届赛事上大放异彩的姆巴佩而言，大巴黎可以吃一颗定心丸，世界杯最佳新秀已经亲自表态：留在巴黎哪里也不去。在接受外媒采访时，姆巴佩表达了继续为巴黎效忠的决心。“我会留在巴黎，和他们一起继续我的路途，我的职业生涯不过刚刚开始”，姆巴佩说道。事实上，在巴黎这座俱乐部，充满了内部的你争我夺。上赛季，卡瓦尼和内马尔因为点球事件引发轩然大波，而内马尔联合阿尔维斯给姆巴佩起“忍者神龟”的绰号也让法国金童十分不爽，为此，姆巴佩的母亲还站出来替儿子解围。而早在二月份，一场与图卢兹的比赛，内马尔也因为传球问题赛后和姆巴佩产生口角。由此可见，巴黎内部虽然大牌云集，但是气氛并不和睦。内马尔离开球队的心思早就由来已久，而姆巴佩也常常与其它俱乐部联系在一起，在躲避过欧足联财政公平法案之后，巴黎正在为全力留下二人而不遗余力。好在姆巴佩已经下定决心，这对巴黎高层而言，也算是任务完成了一半。本届世界杯上，姆巴佩星光熠熠，长江后浪推前浪，大有将C罗、梅西压在脚下的趋势，他两次追赶贝利，一次是在1/8决赛完成梅开二度，另一次是在世界杯决赛中完成锁定胜局的一球，成为不满20岁球员的第二人。另外他在本届赛事中打进了4粒入球，和格列兹曼并列全队第一。而对巴黎而言，他们成功的标准只有一条：欧冠。而留下姆巴佩，可以说在争夺冠军的路上有了仰仗，卡瓦尼在本届世界杯同样表现不错，内马尔虽然内心波澜，但是之前皇马官方已经辟谣没有追求巴西天王，三人留守再度重来，剩下的就是图赫尔的技术战术与更衣室的威望，对图赫尔而言，战术板固然重要，但是德尚已经为他提供了更加成功的范本，像团结法国队一样去团结巴黎圣日耳曼，或许这才是巴黎取胜的钥匙。"
print (docment)

sen_list = docment.strip().split("。")
sen_list.remove("")
doc_list = [jieba.lcut(i) for i in sen_list]
corpus = [" ".join(i) for i in doc_list]

伴随着世界杯的落幕，俱乐部联赛筹备工作又成为主流，转会市场必然也会在世界杯的带动下风起云涌，不过对于在本届赛事上大放异彩的姆巴佩而言，大巴黎可以吃一颗定心丸，世界杯最佳新秀已经亲自表态：留在巴黎哪里也不去。在接受外媒采访时，姆巴佩表达了继续为巴黎效忠的决心。“我会留在巴黎，和他们一起继续我的路途，我的职业生涯不过刚刚开始”，姆巴佩说道。事实上，在巴黎这座俱乐部，充满了内部的你争我夺。上赛季，卡瓦尼和内马尔因为点球事件引发轩然大波，而内马尔联合阿尔维斯给姆巴佩起“忍者神龟”的绰号也让法国金童十分不爽，为此，姆巴佩的母亲还站出来替儿子解围。而早在二月份，一场与图卢兹的比赛，内马尔也因为传球问题赛后和姆巴佩产生口角。由此可见，巴黎内部虽然大牌云集，但是气氛并不和睦。内马尔离开球队的心思早就由来已久，而姆巴佩也常常与其它俱乐部联系在一起，在躲避过欧足联财政公平法案之后，巴黎正在为全力留下二人而不遗余力。好在姆巴佩已经下定决心，这对巴黎高层而言，也算是任务完成了一半。本届世界杯上，姆巴佩星光熠熠，长江后浪推前浪，大有将C罗、梅西压在脚下的趋势，他两次追赶贝利，一次是在1/8决赛完成梅开二度，另一次是在世界杯决赛中完成锁定胜局的一球，成为不满20岁球员的第二人。另外他在本届赛事中打进了4粒入球，和格列兹曼并列全队第一。而对巴黎而言，他们成功的标准只有一条：欧冠。而留下姆巴佩，可以说在争夺冠军的路上有了仰仗，卡瓦尼在本届世界杯同样表现不错，内马尔虽然内心波澜，但是之前皇马官方已经辟谣没有追求巴西天王，三人留守再度重来，剩下的就是图赫尔的技术战术与更衣室的威望，对图赫尔而言，战术板固然重要，但是德尚已经为他提供了更加成功的范本，像团结法国队一样去团结巴黎圣日耳曼，或许这才是巴黎取胜的钥匙。


In [8]:
MMR(doc_list,corpus)

['伴随 着 世界杯 的 落幕 ， 俱乐部 联赛 筹备工作 又 成为 主流 ， 转会 市场 必然 也 会 在 世界杯 的 带动 下 风起云涌 ， 不过 对于 在 本届 赛事 上 大放异彩 的 姆巴 佩 而言 ， 大 巴黎 可以 吃 一颗 定心丸 ， 世界杯 最佳 新秀 已经 亲自 表态 ： 留在 巴黎 哪里 也 不 去',
 '而 留下 姆巴 佩 ， 可以 说 在 争夺 冠军 的 路上 有 了 仰仗 ， 卡瓦 尼 在 本届 世界杯 同样 表现 不错 ， 内马尔 虽然 内心 波澜 ， 但是 之前 皇马 官方 已经 辟谣 没有 追求 巴西 天王 ， 三人 留守 再度 重 来 ， 剩下 的 就是 图 赫尔 的 技术 战术 与 更衣室 的 威望 ， 对图 赫尔 而言 ， 战术 板 固然 重要 ， 但是 德尚 已经 为 他 提供 了 更加 成功 的 范本 ， 像 团结 法国队 一样 去 团结 巴黎 圣日耳曼 ， 或许 这才 是 巴黎 取胜 的 钥匙',
 '好 在 姆巴 佩 已经 下定决心 ， 这 对 巴黎 高层 而言 ， 也 算是 任务 完成 了 一半']